In [1]:
from elasticsearch import Elasticsearch
es = Elasticsearch(['http://localhost:9200'])

Use Elasticsearch to iterate over the reviews and find some opinion units that correspond to the city, topic, and sentiment

In [10]:
sample_size = 1000
found = 0
topic = "Cleanliness"
sentiment = "Bad"
city = "New York"
opinion_units = es.search(index="index_hotels", 
                          scroll="1m", 
                          size=1000, 
                          doc_type="opinion_unit", 
                          body={"query":{"match_all":{}}}
                         )

scroll_id = opinion_units["_scroll_id"]
hits = opinion_units["hits"]["hits"]
rows = []
while found < sample_size:
    for item in hits:
        if topic in item["_source"]["topic"] and sentiment in item["_source"]["sentiment"]:
            review = es.get(index="index_hotels", doc_type="review", id=item["_parent"])
            if city in review["_source"]["city"]:
                rows.append(item["_source"]["content"])
                found += 1
                if found == sample_size:
                    break
    hits = es.scroll(scroll_id=scroll_id, scroll="1m")["hits"]["hits"]

Combine the contents into a single text

In [11]:
text = "\n".join(rows)

Extract keywords

In [12]:
from monkeylearn import MonkeyLearn

ml = MonkeyLearn('<your api key here>')
text_list = [text]
module_id = 'ex_y7BPYzNG'
res = ml.extractors.extract(module_id, text_list)

Print the results!

In [13]:
for d in res.result[0]:
    print d["keyword"], d["relevance"]

room 0.999
bathroom 0.790
carpet 0.407
towels 0.311
bed bugs 0.246
bed 0.232
hotel 0.196
shower 0.155
shared bathroom 0.150
walls 0.138
